In [1]:
import numpy as np
import pandas as pd
import string

In [2]:
train_df = pd.read_csv('data/news_train.txt', sep='\t', header=None, names=['target', 'title', 'content'])
test_df = pd.read_csv('data/news_test.txt', sep='\t', header=None, names=['title', 'content'])

In [3]:
train_df.head()

,target,title,content
0,style,Rolex наградит победителей регаты,Парусная гонка Giraglia Rolex Cup пройдет в Ср...
1,sport,Матс Сундин стал советником тренера сборной Шв...,Шведский хоккеист Матс Сундин назначен советни...
2,media,Брендом года по версии EFFIE впервые стал город,"Гран-при конкурса ""Брэнд года/EFFIE"" получил г..."
3,economics,Цена нефти WTI снизилась после публикации данн...,Цена американской нефти WTI на лондонской бирж...
4,economics,Сбербанк распродаст другим банкирам миллиардны...,"Сбербанк выставил на продажу долги по 21,4 тыс..."


In [4]:
test_df.head()

,title,content
0,В МИД Белоруссии одобрили вынос российского фл...,"Белорусский спортсмен, несший на открытии пара..."
1,Про Хана Соло и Бобу Фетта из «Звездных войн» ...,"Стало известно, кто именно будет центральными ..."
2,В Рунете началась вирусная ICQ-эпидемия,Тысячи пользователей стали жертвой нового ICQ-...
3,Шуховскую башню на Оке признали памятником фед...,Башня конструкции инженера Владимира Шухова в ...
4,Голодец предупредила правительство о последств...,Вице-премьер правительства России Ольга Голоде...


In [5]:
train_df.shape, test_df.shape

((60000, 3), (15000, 2))

In [7]:
sentence = train_df.iloc[5, 2]
sentence

'Многолетний руководитель корпорации Toyota Motor Эйдзи Тоёда (Eiji Toyoda) скончался в возрасте ста лет от сердечного приступа. Об этом сообщает The Japan Daily Press со ссылкой на пресс-службу японской компании.Эйдзи Тоёда был младшим двоюродным братом основателя автоконцерна Киитиро Тоёды (Kiichiro Toyoda). Эйдзи начал работу в компании в 1936 году, а президентом Toyota Motor стал в 1967 году. Он возглавлял концерн в течение 27 лет — дольше, чем кто-либо до него.В 1982 году Тоёда ушел с поста президента компании, заняв кресло председателя совета директоров Toyota. Эту должность он занимал до 1992 года, а спустя еще два года окончательно покинул компанию.Под руководством Эйдзи Тоёда Toyota существенно нарастила продажи автомобилей за рубежом и активно расширяла свое присутствие на зарубежных рынках. Кроме того, с 1983 года он курировал развитие бренда Lexus, люксовой модели Toyota. Также благодаря Эйдзи Тоёде в Toyota была внедрена система максимального эффективного производства авто

In [8]:
table = str.maketrans(string.punctuation, ' '*len(string.punctuation))
sentence.translate(table).lower()

'многолетний руководитель корпорации toyota motor эйдзи тоёда  eiji toyoda  скончался в возрасте ста лет от сердечного приступа  об этом сообщает the japan daily press со ссылкой на пресс службу японской компании эйдзи тоёда был младшим двоюродным братом основателя автоконцерна киитиро тоёды  kiichiro toyoda   эйдзи начал работу в компании в 1936 году  а президентом toyota motor стал в 1967 году  он возглавлял концерн в течение 27 лет — дольше  чем кто либо до него в 1982 году тоёда ушел с поста президента компании  заняв кресло председателя совета директоров toyota  эту должность он занимал до 1992 года  а спустя еще два года окончательно покинул компанию под руководством эйдзи тоёда toyota существенно нарастила продажи автомобилей за рубежом и активно расширяла свое присутствие на зарубежных рынках  кроме того  с 1983 года он курировал развитие бренда lexus  люксовой модели toyota  также благодаря эйдзи тоёде в toyota была внедрена система максимального эффективного производства авто